# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21686499


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:  13%|█▎        | 4/30 [00:01<00:11,  2.19it/s]

Rendering models:  17%|█▋        | 5/30 [00:06<00:45,  1.80s/it]

Rendering models:  23%|██▎       | 7/30 [00:07<00:30,  1.31s/it]

Rendering models:  27%|██▋       | 8/30 [00:07<00:21,  1.00it/s]

...

Rendering models:  87%|████████▋ | 26/30 [00:11<00:00,  5.10it/s]

Rendering models:  90%|█████████ | 27/30 [00:11<00:00,  4.84it/s]

Rendering models:  97%|█████████▋| 29/30 [00:11<00:00,  5.51it/s]

Rendering models: 100%|██████████| 30/30 [00:11<00:00,  5.77it/s]

not-logged-in-f7e2260ed4a9517a3e4a    0.001472
ncaro                                 0.006078
kjkavene                              0.000569
not-logged-in-3e831daf31432a82f496    0.396125
rnevils                               0.000508
bldelacr                              0.002157
mtajnai                               0.000561
ElisabethB                            0.000738
not-logged-in-2e6a81665c2f7bc85757    0.000485
qbright                               0.000595
jmartinez468                          0.006280
pangeli5                              0.011080
peakscience7                          0.000659
tominas                               0.000922
lac515                                0.012004
helios1235                            0.000631
Lavadude                              0.024524
not-logged-in-15498ebaa693dd5a198d    0.000609
not-logged-in-92531cfb2471bb613fa9    0.003629
not-logged-in-d55d56ff31d5a738495f    0.004210
brigi.baumann                         0.002483
h4ranek      

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())